# Music name generator
Well here we are. I'm going to find THE best music name, thanks to machine learning

In [2]:
import numpy as np 
import pandas as pd 
import string
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/home/goznalo/Programming/Python/musicnames'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


%matplotlib inline

/home/goznalo/Programming/Python/musicnames/artists.csv
/home/goznalo/Programming/Python/musicnames/miusic.ipynb
/home/goznalo/Programming/Python/musicnames/.ipynb_checkpoints/miusic-checkpoint.ipynb


### Importing the data
Let's import the data from the kaggle dataset music-artists-popularity, using the Pandas library.

In [30]:
filename = '/home/goznalo/Programming/Python/musicnames/artists.csv'
dataset = pd.read_csv(filename, usecols = [2]) # Obtain the artist name column.

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
dataset.head # preview of the data

<bound method NDFrame.head of                  artist_lastfm
0                     Coldplay
1                    Radiohead
2        Red Hot Chili Peppers
3                      Rihanna
4                       Eminem
...                        ...
1466078                    NaN
1466079                    NaN
1466080                    NaN
1466081                    NaN
1466082                    NaN

[1466083 rows x 1 columns]>

### Clearing, part 1
We don't want any of the NaN entries, nor duplicate names. We will also convert it to a numpy array for posterior transformations.

In [32]:
df = dataset.dropna(axis = 0,how = 'any',thresh = None).drop_duplicates(subset=None) #removing NaN's and converting to numpy.
names = np.asarray(df) 

In [33]:
numnames = names.shape[0]

In [34]:
names[0]

array(['Coldplay'], dtype=object)

### Clearing, part 2
We will now remove all those names using non-standard characters. What I mean by non-standard is all those characters not being latin ones, nor punctuation ones, nor digits, nor spaces. For instance, we will get rid of those using chinese characters or greek letters.

We quickly run intro trouble, as the .isalnum() method applied to these non-latin characters returns True. We need to come up with a solution. One way around this is by using the string module. With it, the string.printable characters are those we want to allow in our names, therefore we make a set out of them, which will be used against the names of the database in a loop.

In [35]:
validchars = set(string.printable)
print(validchars)

{'I', 'E', 'Z', 'o', '%', '!', 'R', ' ', 'y', '}', 'd', 't', '\\', '7', '_', 'S', '6', 'w', 'B', '8', '"', 'h', '4', ':', '\x0b', 'c', '#', 'F', 'z', 'M', '{', 'k', '^', '\t', ',', 'v', 'H', 'G', 'K', ';', 'T', 'p', '5', 'X', 'l', 'U', "'", '=', 'j', '/', 'L', '\r', 'A', '2', 'i', '[', 'Y', 'N', '`', '9', ']', 'q', 'r', ')', '\n', 'e', 'C', '?', 'n', '*', 'O', '0', 'a', 'b', 'J', '\x0c', '(', '-', 'g', '+', 'u', '<', 'Q', 'm', 'f', 'V', 'P', '.', 'W', '>', '@', 's', '|', 'x', '$', '1', '3', '~', 'D', '&'}


In [36]:
testbad = "hello!"
if not all(char in validchars for char in testbad):
    print("invalid")
else:
    print("valid")

valid


Now comes the loop. An initial approach was creating a new object, a numpy array, to which we would append each valid name, therefore discarding the rest of the names with invalid characters. However, the appending operation makes the algorithm take exponential time of completion. Instead, we can save the index of each invalid word in a list, which we then feed to the np.delete() function to remove those entries of the "names" array

In [43]:
#clearlist = np.zeros((1,1))
i = 0 # counts each iteration, i.e. the 
j = 0 # counts each invalid word, used to check every now and then that names are correctly discarded.
deletelist = []
for name in names:
    if not all(char in validchars for char in name[0]):
        deletelist.append(i)
        if j%10000 == 0:
            print("invalid: " + name[0])
        j += 1
    #else:
    #    clearlist = np.append(clearlist, name[0])
    i += 1
    if i%50000 == 0:

        print(str(i) + " cases inspected.")
        
names[deletelist[0]] # check that this corresponds to the first invalid word, Beyoncé

invalid: Beyoncé
50000 cases inspected.
100000 cases inspected.
150000 cases inspected.
invalid: Min Ronnersjö
200000 cases inspected.
250000 cases inspected.
300000 cases inspected.
invalid: Kaszás Attila
350000 cases inspected.
400000 cases inspected.
invalid: E=MC²
450000 cases inspected.
500000 cases inspected.
invalid: Natércia Maria
550000 cases inspected.
600000 cases inspected.
invalid: 堀米ゆず子
650000 cases inspected.
700000 cases inspected.
invalid: Fütumche!
750000 cases inspected.
invalid: Léo Clarens
800000 cases inspected.
850000 cases inspected.
invalid: 권기욱
900000 cases inspected.
invalid: Arbeiter-Chor (Irmler-Männerchor)
950000 cases inspected.


array(['Beyoncé'], dtype=object)

In [47]:
clearedlist = np.delete(names, deletelist)
clearedlist[0:22] # Check that Beyoncé has been correctly removed.

array(['Coldplay', 'Radiohead', 'Red Hot Chili Peppers', 'Rihanna',
       'Eminem', 'The Killers', 'Kanye West', 'Nirvana', 'Muse', 'Queen',
       'Foo Fighters', 'Linkin Park', 'Lady Gaga', 'The Rolling Stones',
       'Daft Punk', 'Green Day', 'Katy Perry', 'The Beatles', 'Oasis',
       'Gorillaz', 'Michael Jackson', 'Maroon 5'], dtype=object)

In [40]:
np.savetxt("clearedlist.csv", clearedlist, delimiter=",") #save the list to a csv file.

### Clearing, part 3: choose your own adventure
We will now distinguish four different cases which can be studied, with decreasing complexity. 
1. The full list, as is.
2. The list, having removed punctuation signs.
3. The list, having removed both punctuation characters and names with more than two words.
4. The list, having removed both punctuation characters and names with more than one word.